In [ ]:
%pip install -q -U --pre triton
%pip install -q -U accelerate==0.12.0 transformers diffusers

In [ ]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
#@markdown Upload your 1 image by running this cell.

import os
from google.colab import files
import shutil

#@markdown Name/Path of the initial model.
MODEL_NAME = "CompVis/stable-diffusion-v1-4" #@param {type:"string"}

#@markdown Enter the directory name to save model at.
OUTPUT_DIR = "/content" #@param {type:"string"}

uploaded = files.upload()
for filename in uploaded.keys():
    INPUT_IMAGE = os.path.join(OUTPUT_DIR, filename)
    shutil.move(filename, INPUT_IMAGE)

In [ ]:
import torch

In [ ]:
%load_ext autoreload
%autoreload 2

from pipeline_stable_diffusion_img2img import StableDiffusionImg2ImgPipeline
from diffusers import DDIMScheduler
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(MODEL_NAME, scheduler=scheduler, torch_dtype=torch.float16).to('cuda')

In [ ]:
from PIL import Image
import torch

num_samples = 4 #@param {type:"number"}
prompt = "zebra" #@param {type:"string"}
diffedit_reference_text = "horse" #@param {type:"string"}
diffedit_noise_ratio = 0.5 #@param {type:"number"}
guidance_scale = 5 #@param {type:"number"}
strength = 0.7 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 4324 #@param {type:"number"}
g_cuda.manual_seed(seed)

init_img=Image.open(INPUT_IMAGE).resize((height, width))
display(init_img)

with torch.autocast("cuda"), torch.inference_mode():
    images = pipe(
        init_image=init_img,
        prompt=prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        strength=strength,
        diffedit_noise_ratio=diffedit_noise_ratio,
        diffedit_reference_text=diffedit_reference_text,
        generator=g_cuda
    ).images

for img in images:
    display(img)